In [1]:
import tensorflow as tf
import requests
import numpy as np
import matplotlib.pyplot as plt
import requests
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.animation import FuncAnimation

from scipy.optimize import curve_fit
from scipy.integrate import quad
from scipy.optimize import curve_fit

from src.plotter.Plotter import Plotter
from src.maths.Error import r_squared
from src.network.Genetic import de

<h2 align='center'>Loading data from server</h2>

In [2]:
data = requests.get('http://v5464.hosted-by-vdsina.com:8000/science/phases/?format=json&limit=1000').json()['results']
temperature = np.array([dat['temperature'] for dat in data], dtype=np.float32)
pressure = np.array([dat['pressure'] for dat in data], dtype=np.float32)
density = np.array([dat['density'] for dat in data], dtype=np.float32)

sort_mask = np.lexsort((temperature, pressure, density))
temperature = temperature[sort_mask]
pressure = pressure[sort_mask]
density = density[sort_mask]


saturation_data = requests.get('http://v5464.hosted-by-vdsina.com:8000/science/saturations/?format=json&limit=81&offset=81').json()['results']
saturation_temperature = np.array([dat['temperature'] for dat in saturation_data])
saturation_pressure = np.array([dat['pressure'] for dat in saturation_data])
saturation_density = np.array([dat['density'] for dat in saturation_data])

sort_mask = np.lexsort((saturation_temperature, saturation_pressure, saturation_density))
saturation_temperature = saturation_temperature[sort_mask]
saturation_pressure = saturation_pressure[sort_mask]
saturation_density = saturation_density[sort_mask]


linearized_temperature = (temperature - min(temperature)) / (max(temperature) - min(temperature))
linearized_pressure = (pressure - min(pressure)) / (max(pressure) - min(pressure))
linearized_density = (density - min(density)) / (max(density) - min(density))


M = 200.592e-3
R = 8.31
N_a = 6e24

critical_temperature = 1490+273.15
critical_pressure = 1510e5
critical_density = 5500

temperature_set = np.array(list(set(temperature)), dtype=np.float32)
temperature_set.sort()

pressure_set = np.array(list(set(pressure)), dtype=np.float32)
pressure_set.sort()

density_set = np.array(list(set(pressure)), dtype=np.float32)
density_set.sort()

In [3]:
%matplotlib qt

<h2 align='center'>Saturation line</h2>

<h3>Plotting the experimental data</h3>

In [76]:
max(saturation_pressure)

125170000.0

In [4]:
plt.rcParams.update({'font.size' : 16})

In [74]:
fig = plt.figure(layout='compressed', figsize=(7, 6))
plt.scatter(saturation_temperature, saturation_pressure, color='b', label='Экспериментальные данные')
plt.scatter(critical_temperature, critical_pressure, color='r', label='Критическая точка')
# plt.grid(True)
plt.xlabel(r'$T_s$ $(K)$', fontsize=20)
plt.ylabel(r'$p_s$ $(Па)$', fontsize=20)
plt.legend(edgecolor='k')
plt.show()

<h3>Plotting the linearized data</h3>

In [20]:
fig = plt.figure(layout='compressed', figsize=(6, 6))
plotter = Plotter(dimension='2d')
plotter.scatter(critical_temperature / saturation_temperature, np.log(saturation_pressure / critical_pressure), color='b', label='Экспериментальные данные')
plotter.scatter(1, 0, color='r', label='Критическая точка')
plotter.xlabel(r'$T_c/T_s$')
plotter.ylabel(r'ln $(p_s/p_c)$')
plotter.legend()
plotter.grid(True)
plotter.show()

<h3>Converting the data to linearized fraction of critical point</h3>

In [24]:
x = lambda t: critical_temperature / t
y = lambda t: np.log(t / critical_pressure)

plotter = Plotter(dimension='2d')
plotter.scatter(x(saturation_temperature), y(saturation_pressure), color='b', label='Экспериментальные данные')
plotter.scatter(x(critical_temperature), y(critical_pressure), color='r', label='Критическая точка')
plotter.grid(True)
plotter.xlabel(r'$T_c/T_s$')
plotter.ylabel(r'$ln(p_s/p_c)$')
plotter.legend(edgecolor='k')
plotter.show()

<h3>Approximating data to linearized fraction of critical point</h3>

In [38]:
def fit_saturation_line(x, a, b):
    return a * x + b


x = lambda t: critical_temperature / t
y = lambda t: np.log(t / critical_pressure)
X = lambda t: critical_temperature / x(t)
Y = lambda t: np.exp(y(t)) * critical_pressure

fit_saturation_line_popt, fit_saturation_line_pcov = curve_fit(fit_saturation_line, x(saturation_temperature), y(saturation_pressure))
fit_saturation_line_popt = np.round(fit_saturation_line_popt, 2)

plt.figure(layout='compressed', figsize=(7, 6))
plotter = Plotter(dimension='2d')
plotter.scatter(x(saturation_temperature), y(saturation_pressure), color='b', label='Экспериментальные данные')
plotter.scatter(x(critical_temperature), y(critical_pressure), color='r', label='Критическая точка')
plotter.plot(
    x(saturation_temperature), fit_saturation_line(x(saturation_temperature), *fit_saturation_line_popt), 
    color='r', label='Уравнение линии насыщения'
)
plt.annotate(
    rf'$y(x)={fit_saturation_line_popt[0]}x+{fit_saturation_line_popt[1]}$', 
    xy=(3.3, -9), xytext=(4, -7.5), 
    arrowprops={'facecolor': 'black', 'shrink' : 0.05}
)
plotter.xlabel(r'$T_c/T_s$', fontsize=20)
plotter.ylabel(r'ln $(p_s/p_c)$', fontsize=20)
plotter.legend(edgecolor='k')
plotter.show()

<h3>Approximating data to fraction of critical point</h3>

In [39]:
def fit_saturation_line(x, a, b):
    return a * x + b


x = lambda t: critical_temperature / t
y = lambda t: np.log(t / critical_pressure)
X = lambda t: t / critical_temperature
Y = lambda t: np.exp(t) * critical_pressure

fit_saturation_line_popt, fit_saturation_line_pcov = curve_fit(fit_saturation_line, x(saturation_temperature), y(saturation_pressure))
saturation_line_linspace_x = np.linspace(min(saturation_temperature), critical_temperature)
saturation_line_linspace_y = fit_saturation_line(x(saturation_line_linspace_x), *fit_saturation_line_popt)

r = r_squared(Y(fit_saturation_line(x(saturation_temperature), *fit_saturation_line_popt)), saturation_pressure)
r = np.round(r, 3)
plt.figure(layout='compressed', figsize=(7, 6))
plotter = Plotter(dimension='2d')
plotter.scatter(saturation_temperature, saturation_pressure, color='b', label='Экспериментальные данные')
plotter.scatter(critical_temperature, critical_pressure, color='r', label='Критическая точка')
plotter.plot(saturation_line_linspace_x, Y(saturation_line_linspace_y), color='r', label='Уравнение линии насыщения, \n$R^2=%r$' %r)
plt.annotate(
    r'$\dfrac {p_s} {p_c}=e^{4.1 \bullet (1- \dfrac {T_c} {T_s})}$', 
    xy=(1300, 0.4e8), xytext=(500, 0.6e8), arrowprops={'facecolor': 'black', 'shrink' : 0.05},
    fontsize=22
)
plotter.xlabel(r'$T_s$ $(K)$', fontsize=20)
plotter.ylabel(r'$p_s$ $(Па)$', fontsize=20)
plotter.legend(edgecolor='k')
plotter.show()
print(r)

0.998


<h3>Attempting water saturation line equation</h3>

In [17]:
def water_equation_of_state(x, a_1, a_2, a_3, a_4, a_5, a_6):
    return x * (a_1 * (1-1/x) + a_2 * (1-1/x) ** 1.5 + a_3 * (1-1/x) ** 3 + a_4 * (1-1/x) ** 3.5 + a_5 * (1-1/x) ** 4 + a_6 * (1-1/x) ** 7.5)

x = lambda t: critical_temperature / t
y = lambda t: np.log(t / critical_pressure)
X = lambda t: t / critical_temperature
Y = lambda t: np.exp(t) * critical_pressure

water_popt, water_pcov = curve_fit(water_equation_of_state, x(saturation_temperature), y(saturation_pressure))
water_equation_of_state_linspace_x = np.linspace(min(saturation_temperature), critical_temperature)
water_equation_of_state_linspace_y = Y(water_equation_of_state(x(water_equation_of_state_linspace_x), *water_popt))

r = r_squared(Y(water_equation_of_state(x(saturation_temperature), *water_popt)), saturation_pressure)
plt.figure(layout='compressed', figsize=(7, 6))
plotter = Plotter(dimension='2d')
plotter.scatter(saturation_temperature, saturation_pressure, color='b', label='Экспериментальные данные')
plotter.scatter(critical_temperature, critical_pressure, color='r', label='Критическая точка')
plotter.plot(water_equation_of_state_linspace_x, water_equation_of_state_linspace_y, '--k', label='Уравнение Вагнера')
plotter.plot(saturation_line_linspace_x, Y(saturation_line_linspace_y), '--r', label='Уравнение линии насыщения')
plotter.grid(True)
plotter.xlabel(r'$T_s$')
plotter.ylabel(r'$p_s$')
plotter.legend(edgecolor='k')
plotter.show()
print(r)

0.9997470798645709


In [72]:
print(f'a_1: {np.abs(water_popt[0])} \n'
      f'a_2: {np.abs(water_popt[1]) ** 1/1.5} \n'
      f'a_3: {np.abs(water_popt[2]) ** 1/3} \n'
      f'a_4: {np.abs(water_popt[3]) ** 1/3.5} \n'
      f'a_5: {np.abs(water_popt[4]) ** 1/4} \n'
      f'a_6: {np.abs(water_popt[5]) ** 1/7.5} \n')

a_1: 0.4038586740483619 
a_2: 7.1628473038264895 
a_3: 26.52998930787577 
a_4: 39.24967506544983 
a_5: 16.659159772464204 
a_6: 0.2804668999844017 


In [16]:
water_popt

array([  -0.40385867,  -10.74427096,   79.58996792, -137.37386273,
         66.63663909,   -2.10350175])

<h3>Plotting gauss expression</h3>

In [195]:
def gauss_expression_of_saturation(x, a_1, a_2):
    return x * (a_1 * (1 - 1 / x) ** 2 + a_2 * (1 - 1 / x) ** 4)

x = lambda t: critical_temperature / t
y = lambda t: np.log(t / critical_pressure)
X = lambda t: t / critical_temperature
Y = lambda t: np.exp(t) * critical_pressure

gauss_popt, gauss_pcov = curve_fit(gauss_expression_of_saturation, x(saturation_temperature), y(saturation_pressure))
gauss_x_linspace = np.linspace(min(saturation_temperature), max(saturation_temperature))
gauss_y_linspace = Y(gauss_expression_of_saturation(x(gauss_x_linspace), *gauss_popt))

fig = plt.figure()
plt.scatter(saturation_temperature, saturation_pressure, color='b', label='Experimental data')
plt.scatter(critical_temperature, critical_pressure, color='r', label='Critical point')
plt.plot(gauss_x_linspace, gauss_y_linspace, '--k', label='Saturation equation')
plt.legend()
plt.grid(True)
plt.show()

In [15]:
gauss_pcov

array([[ 0.11419108, -0.17969957],
       [-0.17969957,  0.29066737]])

<h3>Compare experimental data to ideal gas</h3>

In [5]:
Z = pressure * M / (R * temperature)

plt.figure(layout='compressed', figsize=(6, 6))
plt.scatter(density, Z, s=50, alpha=0.5, color='b', label='Экспериментальные данные')
plt.plot([min(density), max(density)], [min(density), max(density)], 'r', label='Уравнение идеального газа')
plt.grid(True)
plt.xlabel(r'$\rho$ $(\dfrac {кг} {м^3})$')
plt.ylabel(r'$Z = \dfrac {p \mu} {RT}$ $(\dfrac {кг} {м^3})$')
plt.legend()
plt.show()
plt.savefig(r'D:\Учеба\Диплом\Figures\compressibiliy_plotted_ideal.png')

<h2 align='center'>Van-der-Vaals equation of state</h2>

In [40]:
def van_der_vaals(xy, a, b):
    p, d = xy
    return (p + a * d ** 2 / M ** 2) * (M / d - b) / R


van_der_vaals_popt, van_der_vaals_pcov = curve_fit(van_der_vaals, (pressure, density), temperature)

plotter = Plotter(dimension='2d')
plotter.scatter(temperature, van_der_vaals((pressure, density), *van_der_vaals_popt))
plotter.grid(True)
plotter.show()
print(van_der_vaals_popt)

[7.65566499e-01 2.87309402e-05]


In [41]:
a, b = van_der_vaals_popt
V_c = 3 * b
p_c = a / (27 * b ** 2)
T_c = 8 * a / (27 * b * R)
d_c = M / (3 * b)
print(f'k_c = {R * T_c / (p_c * V_c)}')
print(f'd_c = {M / (3 * b)}')
print(f'T_c = {T_c}')

k_c = 2.666666666666667
d_c = 2327.247191802436
T_c = 950.075854000435


In [8]:
def van_der_vaals_DE(xy, a):
    p, d = xy
    return (p + a[0] * d ** 2 / M ** 2) * (M / d - a[1]) / R

def rmse(a):
    pred = van_der_vaals_DE((pressure, density), a)
    return np.sqrt(np.sum(pred - temperature) ** 2 / len(temperature))

result = list(de(rmse, bounds=[(0, 1), (0, 1e-4)], its=100, popsize=50))
args = result[-1][0]

plotter = Plotter(dimension='2d')
plotter.scatter(temperature, van_der_vaals_DE((pressure, density), args))
plotter.xlabel(r'$T_{exp}$')
plotter.ylabel(r'$T_{VDV}$')
plotter.grid(True)
plotter.show()

In [48]:
def reduced_van_der_vaals(xy):
    p, d = xy
    return (p + 3 * d ** 2) * (3 / d - 1) / 8

r = r_squared(reduced_van_der_vaals([pressure / critical_pressure, density / critical_density]), temperature)
r = np.round(r, 4)
plt.figure(layout='compressed', figsize=(6, 5))
plotter = Plotter(dimension='2d')
plotter.scatter(temperature / critical_temperature, reduced_van_der_vaals([pressure / critical_pressure, density / critical_density]), alpha=0.3, color='b', s=50)
plotter.plot(
    [min(temperature / critical_temperature), max(temperature / critical_temperature)],
    [min(temperature / critical_temperature), max(temperature / critical_temperature)],
    color='r', label='$y(x)=x$, $R^2=%s$' %r
)
plotter.xlabel(r'$\tau_{эксп}$', fontsize=20)
plotter.ylabel(r'$\tau_{ВдВ}$', fontsize=20)
plotter.legend()
# plotter.grid(True)
plotter.plot()

In [70]:
density_linspace = np.linspace(min(density), max(density))
pressure_linspace = np.linspace(min(pressure), max(pressure))
density_linspace, pressure_linspace = np.meshgrid(density_linspace, pressure_linspace)
temperature_linspace = reduced_van_der_vaals([pressure_linspace / critical_pressure, density_linspace / critical_density]) * critical_temperature
mask = (temperature_linspace < max(temperature)) & (temperature_linspace > min(temperature))

r = r_squared(reduced_van_der_vaals([pressure / critical_pressure, density / critical_density]) * critical_temperature, temperature)
r = np.round(r, 4)
fig = plt.figure(figsize=(10, 9), layout='compressed')
ax = plt.axes(projection='3d')
ax.scatter(temperature, pressure, density, label='Экспериментальные данные', color='#333333', s=40)
tri = ax.plot_trisurf(
    temperature_linspace[mask], pressure_linspace[mask], density_linspace[mask], cmap='jet', label='Уравнение состояния Ван-дер-Ваальса \n$R^2=%s$' %r
)
ax.set_xlabel(r'$T$ $(K)$', labelpad=12, fontsize=20)
ax.set_ylabel(r'$p$ $(Па)$', labelpad=12, fontsize=20)
ax.set_zlabel(r'$\rho$ $(кг/м^3)$', labelpad=12, fontsize=20)
cbar = plt.colorbar(tri, ax=ax, shrink=0.75, pad=0.15)
cbar.set_label(r'$\rho$ $(кг/м^3)$', fontsize=20)
plt.show()
plt.legend(fontsize=20, loc='best')
plt.savefig(r'D:\Учеба\Диплом\Figures\Van-der-vaals-plotted.png')

In [17]:
%matplotlib qt

In [98]:
len(density_linspace[mask])

1610

In [112]:
fig = plt.figure(figsize=(7, 6), layout='compressed')
ax = plt.axes(projection='3d')
ax.plot_trisurf(temperature, pressure, density, cmap='inferno')
plt.show()

<h2 align='center'>Viral equation of state</h2>

<h3>Plotting second viral coefficient</h3>

In [23]:
def viral_equation(x, a):
    return x + a * x ** 2

B, B_temperature, B_error = [], [], []
for temp in temperature_set:
    mask = (temperature == temp)
    density_temp = density[mask]
    pressure_temp = pressure[mask]
    temperature_temp = temperature[mask]
    Z_temp = pressure_temp * M / (R * temperature_temp)

    if len(Z_temp) < 16:
        continue

    fit = curve_fit(viral_equation, density_temp, Z_temp)
    B.append(fit[0][0])
    B_error.append(r_squared(Z_temp, viral_equation(density_temp, *fit[0])))
    B_temperature.append(temp)

B = np.array(B)
B_temperature = np.array(B_temperature)
index = np.lexsort((B, B_temperature))
B, B_temperature = B[index], B_temperature[index]

In [71]:
plt.figure(layout='compressed', figsize=(7, 6))
plt.scatter(B_temperature, B, color='b', label='Экспериментальные данные')
plt.grid(True)
plt.xlabel(r'$T (K)$')
plt.ylabel(r'$B_2(T)$')
plt.legend()
plt.show()

NameError: name 'B_temperature' is not defined

In [42]:
plt.figure(layout='compressed', figsize=(6, 6))
plotter = Plotter(dimension='2d')
for temp in B_temperature:
    mask = (temp == temperature)
    B_index = (temp == B_temperature)
    B_pressure = viral_equation(density[mask], *B[B_index]) * R * temperature[mask] / M
    plotter.scatter(pressure[mask], B_pressure, color='b', s=50)
plotter.plot([np.min(pressure), np.max(pressure)], [np.min(pressure), np.max(pressure)], color='r')
plt.annotate(
    '$y(x)=x$',
    xy=(1e7, 1e7), xytext=(1.25e7, 0.75e7), arrowprops={'facecolor': 'black', 'shrink' : 0.05}
)
plotter.grid(True)
plotter.legend(['Экспериментальные данные'], edgecolor='k')
plotter.xlabel(r'$p_{эксп}$')
plotter.ylabel(r'$p_{вир}$')
plotter.show()

<h3>Van - der - Vaals second viral coefficient</h3>

In [76]:
def van_der_vaals_second_viral(x):
    return R * critical_temperature / (8 * critical_pressure * x) * (x - 27 / 8 * critical_temperature)

B_temperature_linspace = np.linspace(min(B_temperature), max(B_temperature))
B_linspace = van_der_vaals_second_viral(B_temperature_linspace)
plt.figure(layout='compressed', figsize=(9, 8))
plt.scatter(B_temperature, B, label='Экспериментальные данные', color='b', s=50)
plt.plot(B_temperature_linspace, B_linspace, 'r', label='Вириальный коэффициент Ван - дер - Вальса')
plt.legend(edgecolor='k')
plt.xlabel(r'$T (K)$', fontsize=22)
plt.ylabel(r'$B_2(T)$', fontsize=22)
plt.grid(True)
plt.show()

In [89]:
viral_temperature = np.linspace(min(temperature), max(temperature))
viral_density = np.linspace(min(density), max(density))
viral_temperature, viral_density = np.meshgrid(viral_temperature, viral_density)
viral_pressure = R * viral_temperature / M * viral_equation(viral_density, van_der_vaals_second_viral(viral_temperature))

r = r_squared(R * temperature / M * viral_equation(density, van_der_vaals_second_viral(temperature)), pressure)
r = np.round(r, 4)
plt.figure(layout='compressed', figsize=(9, 8))
ax = plt.axes(projection='3d')
ax.scatter(temperature, pressure, density, label='Экспериментальные данные', s=50)
surface = ax.plot_surface(viral_temperature, viral_pressure, viral_density, cmap='jet', alpha=0.5, label='Вириальное уравнение состояния $R^2=%s$' %r)
ax.set_xlabel(r'$T$ $(K)$', labelpad=10, fontsize=18)
ax.set_ylabel(r'$p$ $(Па)$', labelpad=10, fontsize=18)
ax.set_zlabel(r'$\rho$ $(\dfrac {кг} {м^3})$', labelpad=12, fontsize=18)
plt.colorbar(surface, ax=ax, label=r'$\rho$ $(\dfrac {кг} {м^3})$', shrink=0.75, pad=0.15)
plt.legend(edgecolor='k')
plt.show()